<a href="https://colab.research.google.com/github/saadpolo/ML-Track-technical-lead/blob/master/Datapoint_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium


     |████████████████████████████████| 911kB 2.7MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
#import regex as re
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import csv
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from collections import defaultdict
import time 

In [ ]:
url='https://forum.schizophrenia.com'
#  http://quotes.toscrape.com   https://forum.fractalfuture.net/  https://forum.schizophrenia.com/
options=webdriver.ChromeOptions()


In [ ]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        #print (new_height)
        if new_height == last_height:
            #last_height
            # If heights are the same it will exit the function
            break
        last_height = new_height

        #

In [ ]:
def remove_stopword(text):
    string_wo_sw=[]
    for string in text:
        filtered_sentence =remove_stopwords(string)
        string_wo_sw.append(filtered_sentence)
    return (string_wo_sw)
def convert_lower_case(text):
    lc=np.char.lower(text)
    return (lc) 
def remove_apostrophe(text):
    return np.char.replace(text,"'","")
def remove_single_characters(text):
    new_text = ""
    new_text1 = []
    for string in text:
        words=nltk.word_tokenize(string)
        
        for w in words:
            if len(w)>1:
                new_text=new_text + " " + w
    new_text1.append(new_text)
    return new_text1
def remove_punctuation(text):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        text = np.char.replace(text, i, ' ')
    return text
def stem(text):
    stemmer=PorterStemmer()
    new_text=""
    new_text1=[]
    for string in text:
        words=nltk.word_tokenize(string)
        # print(words)
        for w in words:
            v=stemmer.stem(w)
            new_text=new_text + " " + v
    new_text1.append(new_text)
    return (new_text1)
def preprocess_data(text):
    text = convert_lower_case(text)
    #print (text)
    text = remove_punctuation(text)
    #print(text)
    text = remove_apostrophe(text)
    # print (text)
    text = remove_single_characters(text)
    #print (text)
    # data = convert_numbers(data)
    text = remove_stopword(text)
    # print (text)
    text = stem(text)
    #print (text)
    text = remove_punctuation(text)
    return (text)
    #data = convert_numbers(data)

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # results

In [ ]:
def all_data(url):
    ## driver = webdriver.Chrome(options=options ,executable_path='/opt/anaconda3/lib/python3.7/site-packages/chromedriver')
    from selenium import webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
    wd = webdriver.Chrome('chromedriver',options=options)
    #wd.get("https://www.website.com")
    #print(wd.page_source)  # results
    driver=wd
    driver.implicitly_wait(5)
    driver.get(url)
    #scroll(driver,5)
    tags=defaultdict(int)
    soup_a=bs(driver.page_source, 'lxml')

    datapoints=20

    for tag in soup_a.find_all('span',{'class':'category-name'}, limit=datapoints):
        tags[tag.text]+=1

    links=[]
    topics=[]

    ref = soup_a.find_all('span',{'class':'link-top-line'},limit=datapoints)  # just to narrow down soup so I dont collect bulshit stuff

    for text in ref:
        gold = text.find_all('a')
        for text in gold:
            
            topics.append(text.text)
            a=(text.get('href'))
            b="https"
            if a not in b:
                links.append(a)

            # links.append (text.get('href'))
    print (links)

    comments_list=[[]for i in range(len(links))]
    a=0
    for page in links:
        
        # driver.implicitly_wait(30)
        driver.get(url+page)
        #response=requests.get(url+page)
        scroll(driver,5)
        soup = bs(driver.page_source,'html.parser') #, driver.page_source, 'lxml', response.content, 'html parser'
        for comment in soup.find_all('p'):
            comments_list[a].append(comment.text)
        a+=1
    print (comments_list)
    
    
    with open('comments3.csv', mode='w') as data_file:
        writer = csv.DictWriter(data_file, fieldnames=["text"])
        writer.writeheader()
        writer = csv.writer(data_file) #delimiter=',' , quotechar='"', quoting=csv.QUOTE_ALL) # delimiter=',' , quotechar='"', quoting=csv.QUOTE_ALL
        rows=zip(comments_list)
        for row in rows:
            writer.writerow(row)
    # print (comments_list)

    a=0
    for comm in comments_list:
        c=preprocess_data(comm)
        comments_list[a]=c
        a+=1

    print (comments_list)

    with open('data_file3.csv', mode='w') as data_file:
        writer = csv.DictWriter(data_file, fieldnames=["topic", "link","text"])
        writer.writeheader()
        writer = csv.writer(data_file) #delimiter=',' , quotechar='"', quoting=csv.QUOTE_ALL) # delimiter=',' , quotechar='"', quoting=csv.QUOTE_ALL
        rows=zip(topics, links,  comments_list)
        for row in rows:
            writer.writerow(row)



In [ ]:
pip install porterstemmer


In [ ]:
import nltk
nltk.download('punkt')
import gensim
from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
print(all_data(url))

['/t/sarcosine-reduces-symptoms-in-schizophrenia/21132', '/t/l-theanine-is-effective-in-lessening-positive-symptoms-and-improving-sleep-quality-in-patients-with-schizophrenia/22625', '/t/crisis-intervention-resources/136546', '/t/say-anything-part-lxxxix/200630', '/t/just-purchased-a-new-sofa-and-footstool-for-my-room/201328', '/t/do-you-think-there-will-be-fires/201321', '/t/id-like-to-get-to-know-you/201344', '/t/happy-cake-day-schztuna/201293', '/t/kinda-sad-more-than-just-kinda/201326', '/t/song-of-the-century-tops-the-charts/201335', '/t/what-is-the-coolest-thing-you-own/201163', '/t/the-unmotivated-club/198392', '/t/art-killing-tyranny/201295', '/t/coinciedents-in-business-or-street-names-by-your-area/201338', '/t/frustration/201022', '/t/i-hope-i-wont-have-a-panic-anxiety-attack-today/201343', '/t/what-song-are-you-listening-to-right-now-xiii/196732', '/t/is-not-being-able-to-concentrate-the-way-a-kid-concentrates-on-an-indiana-jones-movie-a-product-of-sz-or-just-the-normal-prod